In [7]:
import numpy as np
import cv2 as cv
import random
import os
import pymysql
from selenium import webdriver
from pyvirtualdisplay import Display
from bs4 import BeautifulSoup

global dir_path
dir_path = '/home/pi/AI/API/UUIDDirectory'

class InputPic:
    before = None
    after = None

    def setimg(self, inputimg):
        os.chdir('/home/pi/AI/API')
        self.before = cv.imread(inputimg, 4)
        self.after = cv.resize(self.before,(1000,300))
        pass 
    
    def getimg(self):
        return self.after
    
    def saveimg(self, name, uuid):
        os.chdir('/home/pi/AI/API/UUIDDirectory/'+ uuid +"/after/")
        cv.imwrite(str(name)+ "_resize.png", self.after)
        pass

    def beforesize(self):
        print('변경 전 사이즈는 ', self.before.shape)
        pass
    
    def aftersize(self):
        print('변경 후 사이즈는 ', self.after.shape)
        pass
    
    def createFolder(directory):
        os.chdir(dir_path)
        os.makedirs(directory)
        os.chdir(dir_path+'/'+directory)
        os.makedirs('before')
        os.makedirs('after')
        pass
    
    def deleteFolder(path):
        os.rmdir(dir_path +'/'+ path + '/before')

    def DBUpdate(obj):
        conn = pymysql.connect(host='aiple.gonetis.com', port=6000 ,user='goaiple2020', password='legoaiple20', db='aiple', charset='utf8')
        try:
            with conn.cursor() as curs:
                sql = "update API_FolderUUID set createDate=json_replace(createDate,'$.data','abc') where UUID=%s;"
                curs.execute(sql, obj)                                  #value = 테이블값, data = 속성값
            conn.commit()
        finally:
            conn.close()

    # 크롤링으로 UUID를 자동으로 가져와서 UUID 값을 UUID 변수에 집어넣는 함수
    def getUUID():
        display = Display(visible=0, size=(800,600))
        display.start()
 
        driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver')
        delay_time=3
        driver.implicitly_wait(delay_time)

        driver.get("http://aiple.gonetis.com")
        c = driver.find_element_by_xpath('//input')
        c.click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser') 
        titles = soup.find(id='uuid')
        global uuid
        uuid = titles.text
        driver.quit()
        display.stop()

def pushUUID() :            #UJIN_uuid 순서 오류_API 생성 이후 수정 해야 함.
    return uuid
            
if __name__ == "__main__":
    pic=[]
    InputPic.getUUID()           # 크롤링으로 UUID를 불러온다
    InputPic.createFolder(uuid)  # 불러온 UUID값으로 폴더를 생성한다

    for i in range(0,3):        # 이미지를 불러와서 객체에 추가하고 opencv로 이미지 사이즈 조절을 한다.
        pic.append(InputPic())
        pic[i].setimg(str(i)+'.png')
        
    for i in range(0,3):        # 이미지를 저장한다.
        pic[i].saveimg(i,uuid)
        
    for i in range(0,3):        # 수정 전 이미지 사이즈 출력
        pic[i].beforesize()
        
    for i in range(0,3):        # 수정 후 이미지 사이즈 출력
        pic[i].aftersize()
        
    InputPic.deleteFolder(uuid) # 수정 전 이미지 폴더 삭제
    InputPic.DBUpdate(uuid)     # DB연동하여 Value 값에 data값 추가
    pass

변경 전 사이즈는  (648, 1152, 3)
변경 전 사이즈는  (648, 1152, 3)
변경 전 사이즈는  (648, 1152, 3)
변경 후 사이즈는  (300, 1000, 3)
변경 후 사이즈는  (300, 1000, 3)
변경 후 사이즈는  (300, 1000, 3)
